## RAG-Retrieval Augmented Generation

### Understanding RAG

In [34]:
sports_text = "Baseball is the best sport and there is a baseball game today at 4pm"
finance_text = "The stock market was down today by 500 points, or approximately 1%"

In [35]:
import boto3
bedrock_runtime = boto3.client(region_name='us-east-1', service_name='bedrock-runtime')

In [36]:
import json

In [37]:
json_request = {'inputText': sports_text}
body = json.dumps(json_request)

In [38]:
response = bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-embed-text-v1')

In [39]:
response_body = json.loads(response.get('body').read())

In [40]:
#response_body['embedding']

In [41]:
type(response_body['embedding'])

list

In [42]:
len(response_body['embedding'])

1536

In [43]:
def embed_text(text):
    json_request = {'inputText': text}
    body = json.dumps(json_request)
    response = bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-embed-text-v1')
    return json.loads(response.get('body').read())['embedding']

In [44]:
# embed_text('hello how are you')

In [45]:
import pandas as pd
data = {'name': ['sports_text', 'finance_text'], 'text': [sports_text, finance_text]}
df = pd.DataFrame(data)

In [46]:
df

,name,text
0,sports_text,Baseball is the best sport and there is a base...
1,finance_text,"The stock market was down today by 500 points,..."


In [47]:
df['embedding'] = df['text'].apply(embed_text)

In [48]:
df

,name,text,embedding
0,sports_text,Baseball is the best sport and there is a base...,"[1.140625, -0.30859375, -0.4921875, -0.1640625..."
1,finance_text,"The stock market was down today by 500 points,...","[0.48632812, 0.84765625, -0.37304688, -0.10107..."


In [49]:
import numpy as np

def cosine_similarity1(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    
    dot_product = np.dot(vector1, vector2)
    mag1 = np.linalg.norm(vector1)
    mag2 = np.linalg.norm(vector2)
    
    return dot_product / (mag1*mag2)
    
def cosine_similarity(vector1,vector2):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitude (norm) of each vector
    magnitude_vector1 = np.linalg.norm(vector1)
    magnitude_vector2 = np.linalg.norm(vector2)

    # Calculate the cosine similarity
    return dot_product / (magnitude_vector1 * magnitude_vector2)    

In [50]:
vector1 = np.array(df['embedding'][0])
vector2 = np.array(df['embedding'][1])

In [51]:
cosine_similarity(vector1, vector2)

0.1782999436199406

In [53]:
query = "How did the stock market perform today?"

In [54]:
prompt_embedding = embed_text(query)

In [56]:
df['prompt_similarity'] = df['embedding'].apply(lambda vector : cosine_similarity(vector, prompt_embedding))

In [57]:
df

,name,text,embedding,prompt_similarity
0,sports_text,Baseball is the best sport and there is a base...,"[1.140625, -0.30859375, -0.4921875, -0.1640625...",0.221955
1,finance_text,"The stock market was down today by 500 points,...","[0.48632812, 0.84765625, -0.37304688, -0.10107...",0.665515


In [60]:
most_similar_text = df.nlargest(1, 'prompt_similarity').iloc[0]['text'] # get the row with largest similarity

In [61]:
full_prompt = f"Answer this question based on the context provided. Here is the question:\n{query} Here is the context: {most_similar_text}"

In [62]:
print(full_prompt)

Answer this question based on the context provided. Here is the question:
How did the stock market perform today? Here is the context: The stock market was down today by 500 points, or approximately 1%


In [63]:
body = json.dumps({'prompt': full_prompt, 'temperature':0})

In [64]:
response = bedrock_runtime.invoke_model(body=body, modelId='meta.llama2-13b-chat-v1')

In [65]:
response_body = json.loads(response.get('body').read())

In [67]:
response_body['generation']

' of the total value of the market.\n\nBased on the context, the answer to the question "How did the stock market perform today?" would be:\n\nThe stock market was down today.'